# Diferenciación automática

En este notebook, veremos una metodología para calcular derivadas numéricamente de forma *exacta*, la **diferenciación automática** o **diferenciación algorítmica**

## Significado de la derivada

¿Para qué sirven las derivadas?

Es una medida de tasas de cambio

**[1]** (i) Supón que quieres calcular raíces de una función $f: \mathbb{R} \to \mathbb{R}$ como lo hemos hecho en un notebook anterior, y tienes una manera de calcular la derivada $f'$. Si calculas $f'(X)$ para un intervalo $X$, ¿qué te da? 

* el conjunto de todas las derivadas en el intervalo $X$ 

¿En qué te ayuda esto qué puedes concluir sobre las raíces? 

* Puedo implementar un método de Newton para intervalos $X$ para encontrar raices en el intervalo. Recordemos el método de Newton 

Sean $f:[a,b] \longrightarrow \mathbb{R}$ una función diferenciable a $[a,b]$ y $x_{0} \in [a,b]$ una condición inicial, definimos la función de iteración como $$x_{n+1} = x_{n} -  \dfrac{f(x_{n})}{f'(x_{n})} = \Phi(x_{n})$$ tal que

$\displaystyle\lim_{x_{n} \rightarrow \infty} \Phi(x_{n}) = x^{*}$ con $x^{*}$ la raíz de $f$

Ahora si extendemos esto a intevalo tenemos lo siguiente

Sean $f:X \longrightarrow \mathbb{R}$ una función diferenciable a $X$ y $x_{0} \in X$, punto medio del intervalos como condición inicial, definimos la función de iteración como $$x_{n+1} = x_{n} -  \dfrac{f(x_{n})}{f'(x_{n})}$$

(ii) ¿Se extiende esto a funciones $f:\mathbb{R^n} \to \mathbb{R}^n$?

* Sí

¿De qué manera?

* Con el método de Newton multidimensional

**[2]** Para una función $f:\mathbb{R} \to \mathbb{R}$, considera un intervalo $X$ donde queremos encontrar el rango $f(X)$. 

(i) Recuerda el teorema del valor medio ("mean value theorem"). Fija un $x_0 \in X$ y utiliza el teorema para escribir una expresión para $f(x)$ para $x \in X$, en términos de la derivada de $f$ en algún punto $\xi \in X$.

Por hipótesis $f\in \mathcal{C}^1$ por el teorema de Taylor, y acontado el residuo (de Lagrange), alrededor de $x_0$ fijo tenemos

$$f(x) = f(x_{0}) + f'(\xi)(x - x_{0})$$



(ii) No conocemos $\xi$. ¿Qué podemos hacer al respecto?

* Tomamos todo el intervalo $X$

Si lo haces, encontrarás otra expresión para el rango $f(X)$, que se conoce como el **mean value form** o **centred form**.

Utilizando la idea de acotar con intervalos, tenemos

$$f(X) = f(x_{0}) + f'(X)(X - x_{0})$$

(iii) Compara los resultados numéricos que obtienes de $f(X)$ con la extensión intervalar usual (lo que vimos en un notebook anterior) y esta versión nueva para distintas funciones. ¿Cuándo es mejor la nueva? 


(iv) Si tienes las dos formas de calcular el rango, ¿cómo puedes combinarlos? Impleméntalo.

* Para ello consideremos la función $f(x) = x^2$ en el intervalo $[-1,2]$ , que sabemos el rango es $f([-1,2]) = [0,4]$,  por otro lado usando la "nueva versión" con $x_0 = 1$

$$f([-1,2]) = f(1) + f'\left([-1,2] \right)([-1,2] - 1) = 1 + [-2,4]*[-2,1] = [-7,5] $$

Con $x_0 = 0$

$$f([-1,2]) = f(0) + f'\left([-1,2] \right)([-1,2] - 0) = [-2,4]*[-1,2] = [-4,8] $$

Notemos que ambos contienen el intervalo solución

Al calcular $f'(X)$ usamos la versión intervalar usual.

In [1]:
using Base.Test

In [2]:
struct Intervalo
    inf::Float64
    sup::Float64
    Intervalo(inf,sup) = inf > sup ? error("No es un intervalo valido") : new(inf,sup)
end

In [3]:
import Base: +, -,*,/

function +(x::Intervalo, y::Intervalo)
    return Intervalo(x.inf + y.inf, x.sup + y.sup)
end

function +(x::Intervalo, c::Real) 
    # Aquí definimos la suma de un Intervalo a un Real
    return Intervalo(x.inf + c, x.sup + c)
end

function +(c::Real, x::Intervalo)
    # Aquí definimos la suma de un Real a un Intervalo
    return Intervalo(x.inf + c, x.sup + c)
end


function -(x::Intervalo, y::Intervalo)
    return Intervalo(x.inf - y.inf, x.sup - y.sup)
end

function -(x::Intervalo, c::Real)
    return Intervalo(x.inf - c, x.sup - c)
end

function -(c::Real, x::Intervalo)
    return Intervalo(c-x.inf, c-x.sup)
end

function *(x::Intervalo, y::Intervalo)
    Intervalo(min(x.inf*y.inf, x.sup*y.inf, x.sup*y.sup, x.inf*y.sup), 
        max(x.inf*y.inf, x.sup*y.inf, x.sup*y.sup, x.inf*y.sup))
end

function *(x::Intervalo, c::Real)
    Intervalo(min(x.inf*c, x.sup*c), max(x.inf*c, x.sup*c))
    
    #return Intervalo(x.inf * c, x.sup * c)
end

function *(c::Real, x::Intervalo)
    Intervalo(min(x.inf*c, x.sup*c), max(x.inf*c, x.sup*c))
end

function /(x::Intervalo, y::Intervalo)
    Intervalo(min(x.inf/y.inf, x.inf/y.sup, x.sup/y.inf, x.sup/y.sup), 
        max(x.inf/y.inf, x.sup/y.inf, x.sup/y.sup, x.inf/y.sup))
end

function /(x::Intervalo, c::Real)
    Intervalo(min(x.inf/c, x.sup/c), max(x.inf/c, x.sup/c))
    
    #return Intervalo(x.inf * c, x.sup * c)
end

function /(c::Real, x::Intervalo)
    Intervalo(min(c/x.inf, c/x.sup), max(c/x.inf, c/x.sup))
end


function sqr(x::Intervalo)
    if x.inf < 0 && x.sup >0
        if abs(x.inf) < abs(x.sup)
            return (0, x.sup^2)
        else
            return (0, x.inf^2)
        end
    elseif x.inf < 0 && x.sup < 0
         return (x.sup^2, x.inf^2)
    else
        return (x.inf^2, x.sup^2)
    end
end

sqr (generic function with 1 method)

In [4]:
sqr(Intervalo(-1,2))

(0, 4.0)

In [5]:
Intervalo(-1,2)^2

Intervalo(-2.0, 4.0)

In [6]:
g(x) = x^2
gp(x) = 2x

gp (generic function with 1 method)

In [7]:
typeof(g)

#g

In [8]:
function Fintervalo(f,fp, x::Intervalo , x0::Real)

    #x0 ∈ [x.inf, x.sup] ? nothing : error("x0 no esta en el intervalos")
    
    f(x0) + fp(x)*(x - x0)
    
end

Fintervalo (generic function with 1 method)

In [9]:
Fintervalo(g,gp,Intervalo(-1,2),1)

Intervalo(-7.0, 5.0)

## Diferenciación automática

De cálculo conocemos reglas para calcular derivadas (a diferencia, por ejemplo, de integrales indefinidas). Resulta que es posible *automatizar* estas reglas, para que la computadora las utilice para calcular derivadas de forma numéricamente exacta (a parte de errores de redondeo). Este conjunto de técnicas se llama la **diferenciación automática** o **diferenciación algorítmica**. En este notebook, veremos las bases de este método.

Nota: Este método *no* utiliza ni diferencias finitas, ni manipulación simbólica. Utiliza las reglas de derivadas provenientes del cálculo, pero de forma *numérica*. Especificamos el valor numérico de una variable, por ejemplo, `a = 3` en donde queremos evaluar la derivada de una función $f$, y ¡el método nos regresará el valor numérico de la derivada $f'(a)$!

**[3]** Supón que tienes dos funciones $f$ y $g$ de $\mathbb{R} \to \mathbb{R}$, cuyas derivadas conoces, y quieres calcular derivadas en un punto $a \in \mathbb{R}$ de combinaciones de estas funciones.

(i) Expande $f$ y $g$ en series de Taylor alrededor de $a$ en términos de la distancia $\epsilon$ desde $a$.
 
 Sea $\epsilon = x -a$

* $f(x) = f(a) + \dfrac{f'(a)(\epsilon)}{1!} + \dfrac{f''(a)(\epsilon)^2}{2!} + \cdots = \displaystyle \sum _{k = 0} ^{\infty} \dfrac{f^{(k)}(a) \epsilon ^k }{k!}$

* $g(x) = g(a) + \dfrac{g'(a)(\epsilon)}{1!} + \dfrac{g''(a)(\epsilon)^2}{2!} + \cdots = \displaystyle \sum _{k = 0} ^{\infty} \dfrac{g^{(k)}(a) \epsilon ^k }{k!} $


(ii) Encuentra series de Taylor para la suma $(f+g)$ y el producto $(f \cdot g)$ en la vecindad de $a$.

* $(f + g)(x) = \displaystyle \sum _{k = 0} ^{\infty}  \left[ f^{(k)}(a)  + g^{(k)}(a) \right]  \left(\dfrac{ \epsilon ^k}{k!}  \right) = \displaystyle \sum _{k =0 }^{\infty} \dfrac{(f + g)^{(k)}(a)\epsilon^{k}}{k!}  $

* $(f \cdot g)(x) = \left( \displaystyle \sum _{k = 0} ^{\infty} \dfrac{f^{(k)}(a) \epsilon ^k }{k!} \right)\left( \displaystyle \sum _{l = 0} ^{\infty} \dfrac{g^{(l)}(a) \epsilon ^l }{l!} \right) = \displaystyle \sum _{k =0 }^{\infty} \dfrac{(f \cdot g)^{(k)}(a)\epsilon^{k}}{k!}  $

(iii) Así, recobre las expresiones ya conocidas (de cálculo) para $(f+g)'(a)$ y $(f \cdot g)'(a)$,
en términos de los valores de las funciones y sus derivadas. Concluye cuál información necesitas de cada función para poder calcular derivadas en el punto $a$ de las combinaciones de las funciones.

* ($f + g)'(a) = f'(a) + g'(a)$, es decir, para suma necesitamos los coeficientes de orden epsilon

* $(f \cdot g)'(a) = f'(a)g(a) + f(a)g'(a)$, para esto necesitamos los coeficentes de orden menor o igual a epsilon de $f$ y $g$ 

Necesitamos que $f$ y $g$ sean $\mathcal{C}^{\infty}$ sin polos en $a$

**[4]** Ahora podemos convertir esto en un método numérico para calcular derivadas, como sigue.

(i) Define un tipo nuevo `Dual` que contiene la información necesaria de una función en el punto $a$. (Dejamos implícito el punto $a$; no lo representamos de forma explícita.) 

(ii) Define las operaciones aritméticas básicas sobre objetos de este tipo, siguiendo las reglas que desarrollamos en la pregunta (3).

(iii) ¿Cuál número `Dual` corresponde con la función identidad $\mathbb{1}: x \mapsto x$ en el punto $a$? Esto representará la "variable independiente" $x$.

* Es `Dual(a, 1)`

(iv) Verifica que si agarras un objeto `xx` de la forma encontrada en (iii) y lo enchufas en una función específica `f(x)` de Julia, arroja el valor de la función y de la derivada.

(v) Así, escribe una función que calcule de forma automática la derivada de una función en un punto dado.

In [10]:
# (i)

struct Dual
    fa::Float64
    fpa::Float64
end

In [11]:
Dual(9,6)

Dual(9.0, 6.0)

In [12]:
function +(x::Dual, y::Dual)
    Dual(x.fa + y.fa, x.fpa + y.fpa)
end

function +(c::Real, y::Dual)
    Dual(c + y.fa, y.fpa)
end

function +(x::Dual, c::Real)
    Dual(x.fa + c, x.fpa )
end


function -(x::Dual, y::Dual)
    Dual(x.fa - y.fa, x.fpa - y.fpa)
end

function -(c::Real, y::Dual)
    Dual(c - y.fa, y.fpa)
end

function -(x::Dual, c::Real)
    Dual(x.fa - c, x.fpa )
end

function *(x::Dual, y::Dual)
    Dual(x.fa * y.fa, x.fpa * y.fa + x.fa * y.fpa )
end

function *(c::Real, y::Dual)
    Dual(c* y.fa,  c * y.fpa )
end

function *(x::Dual, c::Real)
    Dual(x.fa*c, x.fpa*c )
end

* (generic function with 188 methods)

In [13]:
Dual(9,6) + Dual(9,6) 

Dual(18.0, 12.0)

In [14]:
@testset "Pruebas de Dual" begin
    @test Dual(9,6) + Dual(9,6) == Dual(18,12)
    @test Dual(9,6) * Dual(9,6) == Dual(81,108)
    @test Dual(9,6) * Dual(1,0) == Dual(9,6)
    @test Dual(3,1) * Dual(3,1) == Dual(9,6)
end

Test Summary:   | Pass  Total
Pruebas de Dual |    4      4


Base.Test.DefaultTestSet("Pruebas de Dual", Any[], 4, false)

In [15]:
f(x) = x^3 - 2x

f (generic function with 1 method)

In [16]:
f(Dual(2,1))

Dual(4.0, 10.0)

In [32]:
function derivada(f,x0::Real)
    x = f(Dual(x0, 1))
    x.fpa
end

derivada (generic function with 1 method)

In [41]:
@test derivada(f,2) == Dual(4,10).fpa

Test Passed

**[5]** (i) Si tienes una función como `exp`, cuando la aplicas a un número `Dual` que representa a una función $f$ cerca del punto $a$, el resultado debe corresponder a la función $(\exp \circ f)$. Impleméntalo.

(ii) ¿Qué ocurre si derivas `exp(exp(x))` en un punto $a$? ¿Es correcto? ¿Qué se ha implementado (¡de forma automática!) aquí?

In [42]:
import Base: exp

function exp(x::Dual)
    Dual(exp(x.fa), exp(x.fa)*x.fpa)
end

exp (generic function with 11 methods)

In [43]:
exp(Dual(1,1))

Dual(2.718281828459045, 2.718281828459045)

In [44]:
g(x) = x^2 

g (generic function with 1 method)

In [45]:
g(Dual(3,1))

Dual(9.0, 6.0)

In [46]:
@test exp(g(Dual(3,1))) == exp(Dual(9,6))

Test Passed

(ii) ¿Qué ocurre si derivas `exp(exp(x))` en un punto $a$? ¿Es correcto? ¿Qué se ha implementado (¡de forma automática!) aquí?

In [47]:
exp(exp(Dual(1,1)))

Dual(15.154262241479262, 41.193555674716116)

In [48]:
expp(x) = exp(exp(x)) 
derivada(expp,1)

41.193555674716116

**[6]** Para una función $f: \mathbb{R}^n \to \mathbb{R}$, quisiéramos calcular la gradiente $\nabla f$. ¿Cómo podemos extender la diferenciación automática a este caso? Puedes restringir atención a $n=2$ para entender cómo funciona.

**[7]** Ahora que hayamos entendido la idea de la diferenciación automática, podemos echar mano del paquete de Julia `ForwardDiff.jl`, donde hay una implementación muy buena.

Lee el manual del paquete para entender cómo calcular derivadas de funciones $\mathbb{R} \to \mathbb{R}$, gradientes de funciones $\mathbb{R}^n \to \mathbb{R}$, y jacobianos de funciones $\mathbb{R}^n \to \mathbb{R}^m$.

**[8]** Ocupa diferenciación automática para calcular el "centered form" de una función $\mathbb{R} \to \mathbb{R}$.